# Identifying Fraudulent Bank Accounts
Author: Mei Yong <br>
https://github.com/mei-yong/BankFraudDetection
* As part of Know-Your-Customer checks or when a customer applies for a new line of credit
* Based on fraud ring detection logic
    * A group of people who mix and match a set of legitimate identification documents to create fake accounts. e.g. person A and B collaborate to create a new fake person C using person A's mobile and person B's social security number (could also be stolen IDs)
    * Difficult to use outlier analysis to catch these cases because when looking at fake person C on its own, they look perfectly normal as a customer. So when they build a seemingly legitimate credit score and request a huge loan in the future, the bank gives them the loan and they take the money and disappear.

* Used the below tutorial as a foundation
    * https://github.com/neo4j-contrib/gists/blob/master/other/BankFraudDetection.adoc

# Initial Setup

In [1]:
# Import libraries
import pandas as pd
from py2neo import Graph
from IPython.display import Image

In [2]:
# Initialise the graph db
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"
graph = Graph(uri=uri, user=user, password=password)

In [3]:
# Test Neo4j connection
query = "MATCH (n) RETURN COUNT(n) AS numberOfNodes"
graph.run(query).evaluate()

0

In [4]:
# Test connection between Neo4j and CSV file location
query = '''
LOAD CSV WITH HEADERS FROM
"https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv"
AS row WITH row LIMIT 1 RETURN row
'''
graph.run(query).evaluate()

{'First_Name': 'John',
 'Bank_Account': '2.34568E+15',
 'Address_City': 'London',
 'Last_Name': 'Doe',
 'UKID': '345-45-3456',
 'Address_Postcode': 'EC1 123',
 'Credit_Card': '1.23457E+15',
 'Phone_Number': '555-555-5555',
 'ID': '1001',
 'Address_Street': '123 Random Street',
 'Unsecured_Loan': None}

# Build the base graph db

In [5]:
# Create uniqueness constraints & indexes
graph.run("CREATE CONSTRAINT ON (a:AccountHolder) ASSERT a.id IS UNIQUE")
graph.run("CREATE CONSTRAINT ON (t:PhoneNumber) ASSERT t.number IS UNIQUE")
graph.run("CREATE CONSTRAINT ON (id:UKID) ASSERT id.id IS UNIQUE")
graph.run("CREATE CONSTRAINT ON (b:BankAccount) ASSERT b.id IS UNIQUE")
graph.run("CREATE CONSTRAINT ON (c:CreditCard) ASSERT c.id IS UNIQUE")
graph.run("CREATE CONSTRAINT ON (l:UnsecuredLoan) ASSERT l.id IS UNIQUE")

In [6]:
# Create account holder nodes
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv" AS row
MERGE (a:AccountHolder { id: toInteger(row.ID) })
ON CREATE SET
    a.first_name = row.First_Name,
    a.last_name = row.Last_Name    
'''
graph.run(query)

In [7]:
# Create address nodes - would be better with a unique ID probably
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv" AS row
MERGE (a:Address { street: row.Address_Street})
ON CREATE SET
    a.city = row.Address_City,
    a.postcode = row.Address_Postcode    
'''
graph.run(query)

In [8]:
# Create telephone number nodes
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv" AS row
MERGE (:PhoneNumber { number: row.Phone_Number})  
'''
graph.run(query)

In [9]:
# Create UK ID nodes - replace this with more realistic national insurance numbers
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv" AS row
MERGE (u:UKID { id: row.UKID})  
'''
graph.run(query)

In [10]:
# Create bank account nodes
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/bank_accounts.csv" AS row
MERGE (b:BankAccount { id: row.Account_ID})
ON CREATE SET
    b.balance = toFloat(row.Balance)
'''
graph.run(query)

In [11]:
# Create credit card nodes
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/credit.csv" AS row
MERGE (c:CreditCard { id: row.Credit_Card_Num})
ON CREATE SET
    c.limit = toFloat(row.Limit),
    c.balance = toFloat(row.Balance),
    c.expiration_date = row.Expiration_Date,
    c.security_code = row.Security_Code
'''
graph.run(query)

In [12]:
# Create loan nodes
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/loans.csv" AS row
MERGE (l:UnsecuredLoan { id: row.Loan_ID})
ON CREATE SET
    l.balance = toFloat(row.Balance),
    l.apr = toFloat(row.APR),
    l.amount = toFLoat(row.Amount)
'''
graph.run(query)

In [13]:
# Create relationships between account holders and addresses
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv" AS row
MATCH (a:AccountHolder { id: toInteger(row.ID) })
MATCH (address:Address { street: row.Address_Street})
MERGE (a)-[:HAS_ADDRESS]->(address) 
'''
graph.run(query)

In [14]:
# Create relationships between account holders and phone numbers
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv" AS row
MATCH (a:AccountHolder { id: toInteger(row.ID) })
MATCH (t:PhoneNumber { number: row.Phone_Number})  
MERGE (a)-[:HAS_PHONENUMBER]->(t) 
'''
graph.run(query)

In [15]:
# Create relationships between account holders and UK IDs
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv" AS row
MATCH (a:AccountHolder { id: toInteger(row.ID) })
MATCH (u:UKID { id: row.UKID})   
MERGE (a)-[:HAS_UKID]->(u) 
'''
graph.run(query)

In [16]:
# Create relationships between account holders and bank account IDs
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv" AS row
MATCH (a:AccountHolder { id: toInteger(row.ID) })
MATCH (b:BankAccount { id: row.Bank_Account})
MERGE (a)-[:HAS_BANKACCOUNT]->(b) 
'''
graph.run(query)

In [17]:
# Create relationships between account holders and credit card IDs
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv" AS row
MATCH (a:AccountHolder { id: toInteger(row.ID) })
MATCH (c:CreditCard { id: row.Credit_Card})
MERGE (a)-[:HAS_CREDITCARD]->(c) 
'''
graph.run(query)

In [18]:
# Create relationships between account holders and loan IDs
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS
FROM "https://raw.githubusercontent.com/mei-yong/BankFraudDetection/master/data/customers.csv" AS row
MATCH (a:AccountHolder { id: toInteger(row.ID) })
MATCH (l:UnsecuredLoan { id: row.Unsecured_Loan})
MERGE (a)-[:HAS_UNSECUREDLOAN]->(l) 
'''
graph.run(query)

# Query the db

In [19]:
# Find account holders who share more than one piece of legitimate contact information
query = '''
MATCH (accountHolder:AccountHolder)-[]->(contactInformation)
WITH contactInformation,
    count(accountHolder) AS RingSize
MATCH (contactInformation)<-[]-(accountHolder)
WITH collect(accountHolder.id) AS AccountHolders,
    contactInformation, RingSize
WHERE RingSize > 1
RETURN AccountHolders AS FraudRing,
    labels(contactInformation) AS ContactType,
    RingSize
ORDER BY RingSize DESC
'''
results = graph.run(query)

for row in results:
    print(row)

<Record FraudRing=[1003, 1002, 1001] ContactType=['Address'] RingSize=3>
<Record FraudRing=[1002, 1001] ContactType=['PhoneNumber'] RingSize=2>
<Record FraudRing=[1003, 1002] ContactType=['UKID'] RingSize=2>


In [21]:
# Determine the financial risk of a possible fraud ring
query = '''
MATCH (accountHolder:AccountHolder)-[]->(contactInformation)

WITH contactInformation,
    count(accountHolder) AS RingSize
MATCH (contactInformation)<-[]-(accountHolder),
    (accountHolder)-[r:HAS_CREDITCARD|HAS_UNSECUREDLOAN]->(unsecuredAccount)
    
WITH collect(DISTINCT accountHolder.id) AS AccountHolders,
    contactInformation, RingSize,
    SUM(CASE type(r)
        WHEN 'HAS_CREDITCARD' THEN unsecuredAccount.limit
        WHEN 'HAS_UNSECUREDLOAN' THEN unsecuredAccount.balance
        ELSE 0
        END) as FinancialRisk
WHERE RingSize > 1

RETURN AccountHolders AS FraudRing,
    labels(contactInformation) AS ContactType,
    RingSize,
    round(FinancialRisk) as FinancialRisk
ORDER BY 	FinancialRisk DESC
'''
results = graph.run(query)

for row in results:
    print(row)

<Record FraudRing=[1003, 1002] ContactType=['Address'] RingSize=3 FinancialRisk=25387.0>
<Record FraudRing=[1003, 1002] ContactType=['UKID'] RingSize=2 FinancialRisk=25387.0>
<Record FraudRing=[1002] ContactType=['PhoneNumber'] RingSize=2 FinancialRisk=9046.0>
